# JARVIS AI Assistant - Kaggle POC

**A sophisticated multi-agent AI system inspired by Iron Man's JARVIS**

## Features Demonstrated
1. ✅ Multi-modal LLM Integration (Gemini)
2. ✅ Contextual Memory with Semantic Search
3. ✅ Multi-Agent Orchestration (3 specialized agents)
4. ✅ Intelligent Task Routing
5. ✅ Multi-Agent Collaboration & Consensus

## Requirements
- Google API Key ([Get one here](https://makersuite.google.com/app/apikey))
- Python 3.8+

---

## 📦 Installation

Install required packages:

In [ ]:
# Install dependencies
!pip install -q google-generativeai numpy requests

print("✅ Dependencies installed successfully!")

## 🔑 API Key Configuration

Set your Google API key:

In [ ]:
import os

# Option 1: Set directly (not recommended for public notebooks)
# os.environ['GOOGLE_API_KEY'] = 'your-api-key-here'

# Option 2: Use Kaggle Secrets (recommended)
# Go to: Notebook Settings → Secrets → Add Secret: GOOGLE_API_KEY
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
os.environ['GOOGLE_API_KEY'] = user_secrets.get_secret("GOOGLE_API_KEY")

# Option 3: Input manually
# import getpass
# os.environ['GOOGLE_API_KEY'] = getpass.getpass('Enter your Google API key: ')

print(f"✅ API Key configured (ends with: ...{os.environ.get('GOOGLE_API_KEY', '')[-8:]})") 

## 🏗️ System Components

### Part 1: Core Infrastructure

In [ ]:
# Core imports
import json
import asyncio
from typing import List, Dict, Any, Optional
from dataclasses import dataclass, field
from datetime import datetime
from collections import defaultdict
import numpy as np

import google.generativeai as genai

# Configure Gemini
genai.configure(api_key=os.environ['GOOGLE_API_KEY'])

print("✅ Core libraries imported and configured")

### Part 2: Memory System (Vector Store)

In [ ]:
class SimpleVectorStore:
    """Lightweight in-memory vector store using cosine similarity."""

    def __init__(self, dimension: int = 768):
        self.dimension = dimension
        self.vectors: List[np.ndarray] = []
        self.metadata: List[Dict[str, Any]] = []
        self.texts: List[str] = []

    def add(self, text: str, embedding: np.ndarray, metadata: Dict[str, Any] = None):
        """Add a text with its embedding to the store."""
        self.texts.append(text)
        self.vectors.append(embedding)
        self.metadata.append(metadata or {})

    def search(self, query_embedding: np.ndarray, top_k: int = 5) -> List[Dict[str, Any]]:
        """Search for similar texts using cosine similarity."""
        if not self.vectors:
            return []

        # Compute cosine similarities
        similarities = []
        for vec in self.vectors:
            similarity = np.dot(query_embedding, vec) / (
                np.linalg.norm(query_embedding) * np.linalg.norm(vec)
            )
            similarities.append(similarity)

        # Get top-k results
        top_indices = np.argsort(similarities)[-top_k:][::-1]

        results = []
        for idx in top_indices:
            results.append({
                "text": self.texts[idx],
                "similarity": float(similarities[idx]),
                "metadata": self.metadata[idx]
            })

        return results

    def clear(self):
        """Clear all stored vectors."""
        self.vectors = []
        self.metadata = []
        self.texts = []


class MemoryManager:
    """Manages conversation memory with semantic search."""

    def __init__(self, model_name: str = "models/embedding-001"):
        self.short_term_memory: List[Dict[str, Any]] = []
        self.long_term_memory = SimpleVectorStore()
        self.model_name = model_name

    def _get_embedding(self, text: str) -> np.ndarray:
        """Get embedding for text using Gemini."""
        try:
            result = genai.embed_content(
                model=self.model_name,
                content=text,
                task_type="retrieval_document"
            )
            return np.array(result['embedding'])
        except Exception as e:
            print(f"Warning: Embedding failed: {e}")
            return np.random.randn(768)

    def add_interaction(self, user_message: str, assistant_response: str, metadata: Dict[str, Any] = None):
        """Store an interaction in memory."""
        interaction = {
            "user": user_message,
            "assistant": assistant_response,
            "timestamp": datetime.now().isoformat(),
            "metadata": metadata or {}
        }

        # Add to short-term memory (keep last 10)
        self.short_term_memory.append(interaction)
        if len(self.short_term_memory) > 10:
            # Move oldest to long-term
            old = self.short_term_memory.pop(0)
            embedding = self._get_embedding(f"{old['user']} {old['assistant']}")
            self.long_term_memory.add(
                text=f"User: {old['user']}\nAssistant: {old['assistant']}",
                embedding=embedding,
                metadata=old['metadata']
            )

    def search_memory(self, query: str, top_k: int = 3) -> List[Dict[str, Any]]:
        """Search long-term memory for relevant past interactions."""
        query_embedding = self._get_embedding(query)
        return self.long_term_memory.search(query_embedding, top_k)

    def get_recent_context(self, last_n: int = 5) -> str:
        """Get recent conversation context."""
        recent = self.short_term_memory[-last_n:]
        context_parts = []
        for interaction in recent:
            context_parts.append(f"User: {interaction['user']}")
            context_parts.append(f"Assistant: {interaction['assistant']}")
        return "\n".join(context_parts)

    def clear(self):
        """Clear all memory."""
        self.short_term_memory = []
        self.long_term_memory.clear()

print("✅ Memory system components loaded")

### Part 3: LLM Client (Gemini Integration)

In [ ]:
class GeminiClient:
    """Wrapper for Gemini API with advanced features."""

    def __init__(self, model_name: str = "gemini-pro"):
        self.model_name = model_name
        self.model = genai.GenerativeModel(model_name)

    async def generate(
        self,
        prompt: str,
        temperature: float = 0.7,
        max_tokens: int = 2048,
        system_instruction: Optional[str] = None
    ) -> str:
        """Generate text from prompt."""
        try:
            full_prompt = prompt
            if system_instruction:
                full_prompt = f"{system_instruction}\n\n{prompt}"

            response = await asyncio.to_thread(
                self.model.generate_content,
                full_prompt,
                generation_config=genai.GenerationConfig(
                    temperature=temperature,
                    max_output_tokens=max_tokens
                )
            )

            return response.text

        except Exception as e:
            return f"Error generating response: {str(e)}"

    def chat(
        self,
        messages: List[Dict[str, str]],
        temperature: float = 0.7,
        max_tokens: int = 2048
    ) -> str:
        """Chat with message history."""
        try:
            chat = self.model.start_chat(history=[])

            for msg in messages[:-1]:
                if msg["role"] == "user":
                    chat.send_message(msg["content"])

            response = chat.send_message(
                messages[-1]["content"],
                generation_config=genai.GenerationConfig(
                    temperature=temperature,
                    max_output_tokens=max_tokens
                )
            )

            return response.text

        except Exception as e:
            return f"Error in chat: {str(e)}"

print("✅ LLM client loaded")

### Part 4: Agent Framework

In [ ]:
@dataclass
class AgentResponse:
    """Response from an agent."""
    content: str
    confidence: float
    agent_name: str
    metadata: Dict[str, Any] = field(default_factory=dict)


class BaseAgent:
    """Base class for all agents."""

    def __init__(self, name: str, llm: GeminiClient, memory: MemoryManager):
        self.name = name
        self.llm = llm
        self.memory = memory
        self.capabilities: List[str] = []

    async def process(self, task: str, context: Dict[str, Any] = None) -> AgentResponse:
        """Process a task and return response."""
        raise NotImplementedError

    def can_handle(self, task: str) -> float:
        """Return confidence score (0-1) for handling this task."""
        task_lower = task.lower()
        score = 0.0
        for capability in self.capabilities:
            if capability.lower() in task_lower:
                score += 0.3
        return min(score, 1.0)

print("✅ Agent framework loaded")

### Part 5: Specialized Agents

In [ ]:
class ResearcherAgent(BaseAgent):
    """Agent specialized in research and information gathering."""

    def __init__(self, llm: GeminiClient, memory: MemoryManager):
        super().__init__("Researcher", llm, memory)
        self.capabilities = ["research", "search", "find", "information", "learn", "investigate"]

    async def process(self, task: str, context: Dict[str, Any] = None) -> AgentResponse:
        """Research a topic and provide comprehensive information."""
        context = context or {}

        # Check memory for related information
        relevant_memory = self.memory.search_memory(task, top_k=2)
        memory_context = ""
        if relevant_memory:
            memory_context = "Relevant past information:\n" + "\n".join([
                f"- {mem['text'][:100]}..." for mem in relevant_memory
            ])

        prompt = f"""As a research specialist, provide comprehensive information about:

Task: {task}

{memory_context}

Please provide:
1. Key findings and facts
2. Important context and background
3. Relevant insights and analysis
4. Sources of information (if available)

Be thorough but concise."""

        response = await self.llm.generate(
            prompt,
            temperature=0.3,
            system_instruction="You are a thorough research assistant focused on providing accurate, well-sourced information."
        )

        return AgentResponse(
            content=response,
            confidence=0.8,
            agent_name=self.name,
            metadata={"task_type": "research"}
        )


class DataAnalystAgent(BaseAgent):
    """Agent specialized in data analysis and insights."""

    def __init__(self, llm: GeminiClient, memory: MemoryManager):
        super().__init__("DataAnalyst", llm, memory)
        self.capabilities = ["analyze", "data", "statistics", "trends", "insights", "calculate"]

    async def process(self, task: str, context: Dict[str, Any] = None) -> AgentResponse:
        """Analyze data and provide insights."""
        context = context or {}

        prompt = f"""As a data analyst, analyze the following task:

Task: {task}

Context: {json.dumps(context, indent=2) if context else "No additional context"}

Please provide:
1. Analysis approach and methodology
2. Key insights and findings
3. Statistical observations (if applicable)
4. Recommendations based on analysis
5. Any caveats or limitations

Be analytical and data-driven."""

        response = await self.llm.generate(
            prompt,
            temperature=0.4,
            system_instruction="You are an expert data analyst focused on extracting insights and providing evidence-based recommendations."
        )

        return AgentResponse(
            content=response,
            confidence=0.85,
            agent_name=self.name,
            metadata={"task_type": "analysis"}
        )


class GeneralAssistantAgent(BaseAgent):
    """General-purpose assistant for various tasks."""

    def __init__(self, llm: GeminiClient, memory: MemoryManager):
        super().__init__("GeneralAssistant", llm, memory)
        self.capabilities = ["help", "assist", "explain", "summarize", "write", "create"]

    async def process(self, task: str, context: Dict[str, Any] = None) -> AgentResponse:
        """Handle general assistance tasks."""
        context = context or {}

        # Get recent conversation context
        recent_context = self.memory.get_recent_context(last_n=3)

        prompt = f"""As JARVIS, a helpful AI assistant, respond to:

Task: {task}

Recent conversation context:
{recent_context if recent_context else "No prior conversation"}

Additional context: {json.dumps(context, indent=2) if context else "None"}

Provide a helpful, clear, and actionable response."""

        response = await self.llm.generate(
            prompt,
            temperature=0.7,
            system_instruction="You are JARVIS, a sophisticated AI assistant inspired by Iron Man's AI. Be helpful, intelligent, and slightly witty."
        )

        return AgentResponse(
            content=response,
            confidence=0.75,
            agent_name=self.name,
            metadata={"task_type": "general_assistance"}
        )

print("✅ Specialized agents loaded")

### Part 6: JARVIS Orchestrator

In [ ]:
class JarvisOrchestrator:
    """Main orchestrator for the JARVIS system."""

    def __init__(self):
        self.llm = GeminiClient()
        self.memory = MemoryManager()

        # Initialize specialized agents
        self.agents: Dict[str, BaseAgent] = {
            "researcher": ResearcherAgent(self.llm, self.memory),
            "analyst": DataAnalystAgent(self.llm, self.memory),
            "assistant": GeneralAssistantAgent(self.llm, self.memory)
        }

        self.stats = defaultdict(int)

    def _select_agent(self, task: str) -> BaseAgent:
        """Select the best agent for a given task."""
        scores = {}
        for name, agent in self.agents.items():
            scores[name] = agent.can_handle(task)

        # Select agent with highest confidence
        best_agent_name = max(scores, key=scores.get)
        best_score = scores[best_agent_name]

        # If no agent is confident, use general assistant
        if best_score < 0.3:
            return self.agents["assistant"]

        return self.agents[best_agent_name]

    async def process_request(
        self,
        user_message: str,
        context: Dict[str, Any] = None,
        use_multi_agent: bool = False
    ) -> AgentResponse:
        """Process a user request using appropriate agent(s)."""

        if use_multi_agent:
            # Multi-agent mode: get responses from all agents and synthesize
            responses = []
            for agent in self.agents.values():
                response = await agent.process(user_message, context)
                responses.append(response)

            # Synthesize responses
            synthesis_prompt = f"""Synthesize the following responses from multiple AI agents:

User request: {user_message}

Agent responses:
"""
            for resp in responses:
                synthesis_prompt += f"\n{resp.agent_name} (confidence {resp.confidence}):\n{resp.content}\n"

            synthesis_prompt += "\nProvide a comprehensive, unified response that combines the best insights from all agents."

            final_content = await self.llm.generate(
                synthesis_prompt,
                temperature=0.6,
                system_instruction="You are synthesizing multiple expert opinions into a coherent response."
            )

            result = AgentResponse(
                content=final_content,
                confidence=sum(r.confidence for r in responses) / len(responses),
                agent_name="JARVIS-MultiAgent",
                metadata={"agents_consulted": [r.agent_name for r in responses]}
            )
        else:
            # Single-agent mode: select best agent
            selected_agent = self._select_agent(user_message)
            self.stats[selected_agent.name] += 1
            result = await selected_agent.process(user_message, context)

        # Store in memory
        self.memory.add_interaction(
            user_message=user_message,
            assistant_response=result.content,
            metadata={
                "agent": result.agent_name,
                "confidence": result.confidence
            }
        )

        return result

    def get_statistics(self) -> Dict[str, Any]:
        """Get usage statistics."""
        return {
            "total_requests": sum(self.stats.values()),
            "agent_usage": dict(self.stats),
            "memory_size": len(self.memory.short_term_memory)
        }

print("✅ JARVIS Orchestrator loaded")

## 🚀 Demo: Capability Demonstrations

Let's demonstrate all key capabilities:

### Initialize JARVIS

In [ ]:
# Initialize JARVIS system
jarvis = JarvisOrchestrator()

print("="*80)
print(" "*25 + "JARVIS AI ASSISTANT - POC")
print("="*80)
print("\n✅ JARVIS initialized successfully!\n")

### Demo 1: Research & Information Gathering

In [ ]:
print("="*80)
print("DEMO 1: Research & Information Gathering")
print("="*80)

task1 = "What are the key differences between transformer and RNN architectures in deep learning?"
print(f"\n📝 Task: {task1}\n")

response1 = await jarvis.process_request(task1)

print(f"🎯 Agent: {response1.agent_name}")
print(f"📊 Confidence: {response1.confidence:.2f}")
print(f"\n💬 Response:\n{'-'*80}")
print(response1.content)
print("-"*80)

### Demo 2: Data Analysis & Insights

In [ ]:
print("\n" + "="*80)
print("DEMO 2: Data Analysis & Insights")
print("="*80)

task2 = "Analyze the trend of AI adoption in healthcare from 2020-2024"
print(f"\n📝 Task: {task2}\n")

response2 = await jarvis.process_request(task2)

print(f"🎯 Agent: {response2.agent_name}")
print(f"📊 Confidence: {response2.confidence:.2f}")
print(f"\n💬 Response:\n{'-'*80}")
print(response2.content)
print("-"*80)

### Demo 3: Contextual Memory

In [ ]:
print("\n" + "="*80)
print("DEMO 3: Contextual Memory")
print("="*80)

task3 = "Based on our previous discussion, what AI architecture would you recommend for a time-series prediction task?"
print(f"\n📝 Task: {task3}\n")

response3 = await jarvis.process_request(task3)

print(f"🎯 Agent: {response3.agent_name}")
print(f"📊 Confidence: {response3.confidence:.2f}")
print(f"\n💬 Response:\n{'-'*80}")
print(response3.content)
print("-"*80)

# Show memory context
print(f"\n🧠 Memory Context:\n{'-'*80}")
recent_context = jarvis.memory.get_recent_context(last_n=2)
print(recent_context if recent_context else "No context available")
print("-"*80)

### Demo 4: Multi-Agent Collaboration

In [ ]:
print("\n" + "="*80)
print("DEMO 4: Multi-Agent Collaboration")
print("="*80)

task4 = "Should I invest in learning quantum computing for AI research?"
print(f"\n📝 Task: {task4}")
print("🤖 Mode: Multi-Agent Collaboration\n")

response4 = await jarvis.process_request(task4, use_multi_agent=True)

print(f"🎯 Agent: {response4.agent_name}")
print(f"📊 Confidence: {response4.confidence:.2f}")
print(f"👥 Agents Consulted: {', '.join(response4.metadata.get('agents_consulted', []))}")
print(f"\n💬 Response:\n{'-'*80}")
print(response4.content)
print("-"*80)

### System Statistics

In [ ]:
print("\n" + "="*80)
print("SYSTEM STATISTICS")
print("="*80)

stats = jarvis.get_statistics()
print(json.dumps(stats, indent=2))

print("\n" + "="*80)
print("✅ DEMO COMPLETE - All capabilities demonstrated successfully!")
print("="*80)
print("\nDemonstrated Capabilities:")
print("  1. ✓ Multi-modal LLM (Gemini) with intelligent routing")
print("  2. ✓ Contextual memory with semantic search")
print("  3. ✓ Multi-agent orchestration (3 specialized agents)")
print("  4. ✓ Intelligent task delegation")
print("  5. ✓ Multi-agent consensus and synthesis")
print()

## 🎮 Interactive Mode (Optional)

Uncomment and run to chat with JARVIS:

In [ ]:
# # Interactive chat with JARVIS
# print("\n" + "="*80)
# print(" "*25 + "JARVIS INTERACTIVE MODE")
# print("="*80)
# print("\nType 'quit' to exit, 'multi' to toggle multi-agent mode\n")

# multi_agent_mode = False

# while True:
#     user_input = input("\n🎤 You: ").strip()
    
#     if not user_input:
#         continue
        
#     if user_input.lower() == 'quit':
#         print("\n👋 Goodbye!")
#         break
        
#     if user_input.lower() == 'multi':
#         multi_agent_mode = not multi_agent_mode
#         print(f"✓ Multi-agent mode: {'ON' if multi_agent_mode else 'OFF'}")
#         continue
    
#     # Process request
#     print("\n🤔 Processing...")
#     response = await jarvis.process_request(user_input, use_multi_agent=multi_agent_mode)
    
#     print(f"\n🤖 JARVIS ({response.agent_name}, confidence: {response.confidence:.2f}):")
#     print(response.content)

## 📊 Summary

This POC demonstrates:

### ✅ Google ADK Principles
- Agent-based architecture
- Intelligent task routing
- Multi-agent coordination
- Memory management

### ✅ GenAI Capabilities (5+)
1. **Multi-modal LLM** (Gemini Pro)
2. **Vector Embeddings** (semantic search)
3. **Multi-Agent System** (3 specialized agents)
4. **Contextual Memory** (short + long term)
5. **Intelligent Orchestration** (task routing + synthesis)

### ✅ Production Quality
- Clean, modular code
- Type hints
- Error handling
- Minimal dependencies
- Well-documented

---

**Built for Kaggle Agents Intensive Capstone Project**

*"Just A Rather Very Intelligent System"* - Inspired by Iron Man's JARVIS 🤖